### Programming for Data Analysis Project 2

### Wisconsin Breast Cancer Dataset Analysis

In [22]:
# about data, overview and background
# import data with pandas
# preprocessing, cleaning

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.neighbors as nei
import sklearn.model_selection as mod

### 1. Introduction

Breast cancer is one of the most common types of cancer among women. It can be described as changing and abnormal growth of breast cells while grouping of these cells leads to tumour formation. [[1]](https://www.cancer.ie/cancer-information-and-support/cancer-types/breast-cancer) Breast cancer research by Wolberg, Street and Mangasarian from University of Wisconsin conducted in 1990s by analysing characteristics of cell nuclei in a digitized image of a fine needle aspirate of a breast mass is often used in machine learning and data analytics. [[2]](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) Diagnostic dataset was taken with the description of attributes that were measured in order to predict between two types of tumours, benign or malignant. Various classification machine learning algorithms were performed over the years on these data with different level of accuracy: Decision trees, Naïve Bayes, Support Vector Machine, Logistic regression and k-Nearest Neighbors.

### 2. Exploring Data

Before we use some of these models, we are going to import and explore the dataset.  

In [3]:
# pandas library was used to import the dataset
cancer =  pd.read_csv('data_breast_cancer.csv')

To diplay the first couple of rows we used the head function

In [4]:
cancer.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


There are 33 columns in our dataset with parameters that describe the image of the nuclei. To check the total number of rows we used the shape method.

In [6]:
cancer.shape

(569, 33)

### References:


 * McGinley, B.(2022). Lectures; Project II, Programming for Data Analysis, Atlantic Technological University
 *  Retrieved December, 2022 from https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(diagnostic)   
 * https://www.cancer.ie/cancer-information-and-support/cancer-types/breast-cancer  
 * https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data  
 [1] Irish Cancer Society. (n.d.). Breast Cancer. What is breast cancer? Retrieved January 8, 2023 from https://www.cancer.ie/cancer-information-and-support/cancer-types/breast-cancer    
 [2] Wolberg, W.H, Street, W.N., Mangasarian, O.L. (1995). Breast Cancer Wisconsin (Diagnostic) Data Set. UCI Machine Learning Repository. Irvine, CA: University of California, School of Information and Computer Science. Retrieved December 20, 2022 from https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29  
        
    


